In [6]:
import config
from sgqlc.operation import Operation
from github_schema import github_schema
from sgqlc.endpoint.http import HTTPEndpoint

headers = {"Authorization": f"bearer {config.GITHUB_API_TOKEN}"}
endpoint = HTTPEndpoint(config.GITHUB_API_URL, headers)

pr_json_list = []

after_cursor = None
while True:
    query = Operation(github_schema.Query)
    pull_requests = query.repository(owner="vuejs", name="vue").pull_requests(first=100, after=after_cursor)

    pull_requests.nodes().id()
    pull_requests.nodes().number()
    pull_requests.nodes().title()
    pull_requests.nodes().repository().name()
    pull_requests.nodes().url()
    pull_requests.nodes().author().login()
    pull_requests.nodes().state()
    pull_requests.nodes().changed_files()
    pull_requests.nodes().body_text()
    pull_requests.nodes().commits(first=100).nodes().commit().message()
    pull_requests.nodes().closed()
    pull_requests.nodes().merged()
    pull_requests.nodes().merged_at()
    pull_requests.nodes().created_at()
    pull_requests.nodes().closed_at()

    pull_requests.page_info.__fields__(end_cursor=True, has_next_page=True)

    data = endpoint(query)

    repo = (query + data).repository

    for pr in repo.pull_requests.nodes:
        pr: github_schema.PullRequest
        if pr.state == 'MERGED':
            pr_json_list.append(pr.__to_json_value__())

    if not repo.pull_requests.page_info.has_next_page:
        break

    after_cursor = repo.pull_requests.page_info.end_cursor

In [ ]:
import pandas as pd
df = pd.DataFrame.from_records(pr_json_list)

def get_login(login_dict):
    try:
        return login_dict["login"]
    except:
        return None

df["author"] = df["author"].apply(lambda item: get_login(item))
df["repository"] = df["repository"].apply(lambda item: item["name"])
df["commits"] = df["commits"].apply(lambda item: [commit["commit"]["message"] for commit in item["nodes"]])

for index, row in df.groupby(["author"]).size().reset_index(name='counts').iterrows():
    if row["counts"] >= 10:
        print(row["author"])



df

In [ ]:
df.loc[df['author'] == "Hanks10100"]

In [29]:
for index, row in df.loc[df['author'] == "Hanks10100"].iterrows():
    print(row["commits"])

['be able to parse the real tag name for the specific platform']
['[weex] expose version and config on subVue']
['[weex] fix the text node could not be removed properly']
['[weex] fix the removeChild logic for text node']
['[weex] use weex.requireModule instead of weex.require', '[weex] fix the deprecated __weex_require_module__']
['Support to delete array item in Vue.delete', 'add test case for Vue.delete array', 'add blank line between test cases']
['[weex] add test case for camelize props', "Merge branch 'dev' into camelcase", '[weex] add test case for append props', '[weex] camelize component attribute name', "Merge branch 'append-tree' into release", '[weex] more reliable camelize functions']
['[weex] Support unary and left open tags', '[weex] add test case for unary tag', '[compiler] move canBeLeftOpenTag to compiler option']
["compile bundle on native side if 'compileBundle()' is available on\nnative side.", "Merge pull request #1 from brunoduan/android-feature-20170321\n\ncompi